In [ ]:
pip install pypdf

In [ ]:
from pypdf import PdfReader

In [ ]:
def extract_text(path):
  reader = PdfReader(path)
  text = ""
  for page in reader.pages:
    text += page.extract_text() + "/n"
  return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text = extract_text("/content/Looters and grabbers.pdf")

In [ ]:
#Why PDFs must be converted to text
#this is because they are designed for visual presentation  rather than machine readability

In [ ]:
#chunking
def chunk_text(text, chunk_size =900, overlap = 200):
  words = text.split()
  chunks = []
  for i in range(0, len(words), chunk_size-overlap):
    chunks.append(" ".join(words[i:i+chunk_size]))
  return chunks

In [ ]:
# Why chunking is necessary
#chunking makes texts smaller making models to retrive relevant information more efficiently

#Why overlap improves retrieval



In [ ]:
#Embeddings
from sentence_transformers import SentenceTransformer

In [ ]:
model = model = SentenceTransformer('BAAI/bge-small-en-v1.5')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
chunks = chunk_text(text)

In [ ]:
book_embedding = model.encode(chunks).tolist()

In [ ]:
len(book_embedding)

343

In [ ]:
# What embeddings represent
#-represents data as numerical vectors
#-captures semantic rlship between data
# Why semantic search is better than keyword search
#-semantic understands meaning



Installing chromadb

In [ ]:
pip install chromadb


In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path = "./content/Looters and grabbers.pdf")

In [ ]:
collection = client.get_or_create_collection(
    name ="book_rag"
)

In [ ]:
ids = [f"book_{i}" for i in range(len(chunks))]

In [ ]:
# adduing chunks and book_id
collection.add(
    documents= chunks,
    embeddings= book_embedding,
    ids = ids

)

Why is metadata important:
1. it improves efficiency allowing users to locate specific documents in a large repository


What happens if embeddings are not stored?
1.

Query Retrieval


In [ ]:
def retrieve(query, top_k=3):
  query_embedding = model.encode(query).tolist()

  results = collection.query(
      query_embeddings=[query_embedding],
      n_results=top_k

  )
  return results["documents"][0]

In [ ]:
results =retrieve("what does the book say about Kenyatta")

In [ ]:
results

['Fernandes, Cyprian. Kindle Edition, Loc. pp. 1285-1301. 140 Macharia, Kinuthia. “Slum clearance and the informal economy in Nairobi,” The Journal of Modern African Studies , Vol. 30, No. 2, June 1992, p. 229. 141 Good, Kenneth. “Kenyatta and the Organization of KANU,” Canadian Journal of African Studies , No. 2, Autumn, 1968, p. 135. See also Daily Nation , 7 December 1964, p. 1. 142 Marchetti, Victor. “Propaganda and Disinformation: How the CIA manufactures history,” Institute for Historic Review, 143 “This is where Jomo used to hold secret meetings – Mama Gathoni Koinange, Metro, undated. 144 Nicholls, Christine. “What Brought Lord Delamere to Kenya?” Old Africa – Stories from East Africa’s Past, 4 August 2014. 145 Mutunga, Kamau. “Mbiyu Koinange, the powerful force behind the Kenyatta presidency,” Daily/nNation , online, 29 October 2011, 146 Meredith, Martin. The Fate of Africa: A history of the continent since independence, Public Affairs, Online, p. 185. 147 Letter to Margaret, 